# Clase 11: Procesamiento de datos espaciales 

- Rasterio: https://rasterio.readthedocs.io/en/latest/quickstart.html
- Geopandas
    - https://pygis.io/docs/a_intro.html
    - Datasets https://www.ign.gob.ar/NuestrasActividades/InformacionGeoespacial/CapasSIG

## Raster

### Con un DEM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


import rasterio as rio
from rasterio.plot import show

In [ ]:
dem = rio.open("datos/geo/ASTGTM2_S32W061_dem.tif")

In [ ]:
dem.name

In [ ]:
dem

In [ ]:
dem.count

In [ ]:
dem.shape

In [ ]:
dem.width

In [ ]:
dem.dtypes


In [ ]:
dem.bounds

In [ ]:
dem.transform

In [ ]:

#dlon = dlat = 0.0002777777777777778
lon = np.linspace(-61.00013888888889,-59.999861111111116,3601)
lat = np.linspace(-32.00013888888889,-30.999861111111112,3601)
len(lat)

In [ ]:
dem.transform * (0,0)

In [ ]:
dem.transform*(3601,3601)

In [ ]:
dem.crs

In [ ]:
dem.indexes # nos devuelve el índice de cada banda: ojo, arrancan de 1

In [ ]:
band1 = dem.read(1) # leemos la banda 1 de la imagen (es la única banda)

In [ ]:
type(band1)

In [ ]:
band1

In [ ]:
plt.imshow(band1, cmap='viridis')

In [ ]:
px.imshow(band1, x=lon, y=lat[::-1])

In [ ]:
show(band1, cmap='gist_earth', transform=dem.transform);

### Con una imagen Landsat

In [ ]:
imglandsat_b1 = rio.open("datos/geo/landsat/LC08_L1TP_227081_20191012_20191018_01_T1/LC08_L1TP_227081_20191012_20191018_01_T1_B1.TIF")

In [ ]:
imglandsat_b1.indexes

In [ ]:
datos_b1 = imglandsat_b1.read(1)
datos_b1

In [ ]:
imglandsat_b1.bounds

In [ ]:
imglandsat_b1.crs
# https://epsg.org/crs_32620/WGS-84-UTM-zone-20N.html

In [ ]:
imglandsat_b1.transform

In [ ]:
imglandsat_b1.width

In [ ]:
imglandsat_b1.height

In [ ]:
imglandsat_b1.transform * (0, 0)

- Vamos a leer todas las bandas de la imagen: veamos el archivo de metadatos

In [ ]:
bandas = 'B1 B2 B3 B4 B5 B6 B7 B8 B9 B10 B11'.split()

imagenes = []

for banda in bandas:
    nom_archi = f'datos/geo/landsat/LC08_L1TP_227081_20191012_20191018_01_T1/LC08_L1TP_227081_20191012_20191018_01_T1_{banda}.TIF'
    img = rio.open(nom_archi)
    imagenes.append(img)

In [ ]:
imagenes

### Calculamos el NDVI

$$\text{NDVI}=\frac{NIR - R}{NIR + R}$$

In Landsat 4-7, $$\text{NDVI} = \frac{Band 4 - Band 3}{Band 4 + Band 3}$$.

In Landsat 8-9, $$\text{NDVI} = \frac{Band 5 - Band 4}{Band 5 + Band 4}$$.

In [ ]:
banda4 = imagenes[3].read(1).astype('float64')
banda5 = imagenes[4].read(1).astype('float64')
banda5[5000,5000]

In [ ]:
nir = banda5
red = banda4

In [ ]:
nume = nir - red
deno = red + nir

# >>>> ignoramos division por cero o valores inválidos <<<<
np.seterr(divide = "ignore", invalid="ignore")

ndvi = nume/deno

In [ ]:
ndvi[ndvi > 1] = np.nan
ndvi[ndvi < -1] = np.nan

In [ ]:
show(ndvi, cmap='Greens')

In [ ]:
# Plot raster
plt.imshow(ndvi)
plt.title("NDVI")
plt.show()

In [ ]:
#export ndvi image
ndviImage = rio.open('ndviImage.tiff', 'w', driver='Gtiff',
                          width=imagenes[3].width, height=imagenes[3].height,
                          count=1,
                          crs=imagenes[3].crs,
                          transform=imagenes[3].transform,
                          dtype='float64'                  
                         )
ndviImage.write(ndvi,1) #ndvi
ndviImage.close()

In [ ]:
np.geterr()